# Project 3 - Web APIs & Classification
_Author: Ritchie Kwan_

---


## Table of Contents

0. [Problem Statement, Assumptions, Executive Summary](#Problem-Statement)
1. [Data Collection](#Data-Collection)
1. [Data Cleaning & EDA](02-Data-Cleaning-and-EDA.ipynb#Data-Cleaning-and-EDA)
1. [Benchmark Model](03-Benchmark-Model.ipynb#Benchmark-Model)
1. [Model Tuning](04-Model-Tuning.ipynb#Model-Tuning)
1. [Evaluation and Conceptual Understanding](04-Model-Tuning.ipynb#Evaluation-and-Conceptual-Understanding)


## Problem Statement

Spam and ham are typically used to describe junk and relevant emails, respectively. In the context of Reddit, any post unrelated to the subreddit can be considered "spam", while posts related to the subreddit are "ham". The integrity of a subreddit relies on the quality and relevance of its posts. The content of a spam post could be as bad as a hyperlink to a phishing website or as benign as a high-quality post accidentally submitted to the wrong subreddit. 

A spam detection bot could be trained to automatically detect and remove irrelevant posts before they are released to the public. **The objective is to determine how the frequency of certain two-word phrases (bi-grams) in a post affect the odds that the post belongs to a specific subreddit by using Neuro-Linguistic Programming (NLP) techniques.**


## Assumptions

The default page of each subreddit is sorted by `hot`, typically posts that have already acquired a sufficient amount of upvotes by users and possibly passed a spam detection bot. Since the posts collected using the Reddit API appear to be sorted the same way, I assume that **all posts collected using the Reddit API are ham**.

## Executive Summary

**Data Collection**  
I collected as many posts from `/r/leanfire` and `/r/fatfire` as Reddit would allow. In addition to `title` and `selftext`, I also collected `comments` to train my models. Since I collected more posts from `/r/leanfire`, it will be my target class `y=1` (ham). Posts from `/r/fatfire` will be used to imitate spam for the sake of training classification models. 

**Data Cleaning and EDA**  
I extracted text from the `title`, `selftext`, and `comments` columns to use as my predictors. I used `RegExpTokenizer` to split the text into individual words and `Lemmatizer` to reduce words to their lemma. 

**Benchmark Model**  
I train test split my data to check for overfitting. I used `CountVectorizer` and `TfidfVectorizer` to map bi-grams to their frequency or weighted frequency in each post. For my benchmark model, I used `LogisticRegression`. For the training data, I used **cross validated accuracy score to measure success** of the model. Not only did the model perform better using the `TfidfVectorizer` transformed data, the highest ranked bi-grams were also more meaningful. 

**Model Tuning**  
I used the following classification models with and without tuning hyperparameters:
- LogisticRegression
- KNN
- NaiveBayes
    - MultinomialNB
- DecisionTreeClassifier
- BaggingClassifier
- RandomForestClassifier
- ExtraTreesClassifier
- AdaBoostClassifier
- SVC
- VotingClassifer

With and without hyperparametering tuning, Naive Bayes' `MultinomialNB` performed the best.

**Evaluation and Conceptual Understanding**  
Unseen data will be newly submitted posts on `/r/leanfire`, meaning all new data belongs to the positive class (`y=1`). If the model classifies a post as negative (`y=0`), then it is likely to be spam.  

True Positives = Ham  
False Negative = Spam  

To maintain the integrity of the subreddit, I prefer the model to have a **sufficiently high False Negative rate** (spam detection rate). The consequence of this decision is that **some quality posts may be blocked**. This is preferred over **letting actual spam bypass the detection algorithm**. 


### Import Libraries

In [18]:
import requests
import time

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Data Collection


### Choose Subreddits

In [19]:
subs = ['fatfire', 'leanfire']
# subs = ['lifeprotips', 'shittylifeprotips']

possible_subs = [('animalsbeingbros', 'animalsbeingjerks'), 
                 ('powerlifting', 'weightlifting'),
                 ('warriors', 'lakers'),
                 ('funny', 'cute'),
                 ('lifeprotips', 'shittylifeprotips'),
                 ('news', 'politics'),
                 ('aww', 'lookatmydog') ]

### Define a function to get all comments in a reddit post

In [20]:
def get_comments(children):
    '''
    Arguments: 
    children : list of child comment JSONs.
    
    Recursively extracts all of the comments of a Reddit post.
    
    Return:
    comments : List of all comments in a reddit post.
    '''
    comments = []
    
    for child in children:
        if child['kind'] == 'more':
            continue
        
        data = child['data']

        comments.append(data['body'])        
        # if child has children, get comments of children
        if data['replies'] != '':
            # get_comments returns a list, so .extend() it
            comments.extend(get_comments(data['replies']['data']['children']))
    
    return comments

## Hit the Reddit API

### Keys to note:  
`permalink` : to get comments  
`title` : predictor words  
`selftext` : more predictor words  
`body` : body of comment

### Define a function to collect the posts and comments of a subreddit

In [21]:
def scrape(subreddit):
    '''
    Arguments:
    subreddit : String of a subreddit
        example: 'leanfire'
    
    Requests as many posts from Reddit as possible
    Stores the text of every post's title, selftext and comments

    Return:
    A dataframe of post titles, selftext and comments
    '''
    headers = {'User-agent' : 'Reechee'}
    subreddit_url = 'https://www.reddit.com/r/' + subreddit + '.json'
    posts = []
    after = ''
    
    # Keep requesting JSONs until Reddit gives up
    while after != None:        
        # Request subreddit JSON for next 100 posts
        req = requests.get(subreddit_url,
                                      headers = headers,
                                      params = {'after' : after,
                                                'limit' : '100'})
        subreddit_json = req.json()
        
        # dictionary of post data
        post_data = subreddit_json['data']
        
        n_posts = len(post_data['children'])
        
        # For each post webpage, request the JSON
        for i, post in enumerate(post_data['children']):
            post_url = 'https://www.reddit.com' + post['data']['permalink'] + '.json'
            
            # Request the post webpage's JSON
            post_req = requests.get(post_url, 
                                     headers = headers)
            post_json = post_req.json()

            # Get all comments of a post
            post['data']['comments'] = get_comments(post_json[1]['data']['children'])
            
            # Print progress
            print('[{}/{}]{} comments from {}'
                  .format(i+1, n_posts,
                  len(post['data']['comments']),
                  post['data']['permalink']))
        
        # extend list of posts
        posts.extend([post['data'] \
                         for post in post_data['children']])
        
        # 'after' key for next batch of posts
        after = post_data['after']
        
        print(f'Fetching next 100 posts with after key: {after}')
    
    print('Done!')
    return pd.DataFrame(posts)



In [22]:
df0 = scrape(subs[0])

[1/100]34 comments from /r/fatFIRE/comments/a815p8/advisor_fees_at_various_asset_levels/
[2/100]37 comments from /r/fatFIRE/comments/a7s721/hsa_record_keeping/
[3/100]93 comments from /r/fatFIRE/comments/a7f38l/2019_money_goals/
[4/100]9 comments from /r/fatFIRE/comments/a7fujr/whats_is_your_tax_loss_harvesting_threshold/
[5/100]87 comments from /r/fatFIRE/comments/a76bub/might_be_retiring_earlier_than_planned_hoping_my/
[6/100]20 comments from /r/fatFIRE/comments/a78hw5/anyone_plan_to_spend_their_last_dollar_on_their/
[7/100]9 comments from /r/fatFIRE/comments/a7632z/such_thing_as_too_much_pretax_investing/
[8/100]75 comments from /r/fatFIRE/comments/a70p6r/anyone_spending_like_an_aggressive_leanfire_but/
[9/100]8 comments from /r/fatFIRE/comments/a72jmq/startup_life_final_update/
[10/100]18 comments from /r/fatFIRE/comments/a6vjjl/deferred_longevity_annuities_to_hedge_longevity/
[11/100]74 comments from /r/fatFIRE/comments/a6t02p/young_with_large_growing_income_where_should_i/
[12/10

[90/100]22 comments from /r/fatFIRE/comments/9wly1m/cant_make_a_decision_ever_wealth_advisor_or_dump/
[91/100]75 comments from /r/fatFIRE/comments/9w7qkl/any_corporate_people_here_that_have_been_able_to/
[92/100]12 comments from /r/fatFIRE/comments/9vzgas/looking_to_get_involved_in_a_different_business/
[93/100]88 comments from /r/fatFIRE/comments/9vld0x/farmland/
[94/100]98 comments from /r/fatFIRE/comments/9v3td8/what_to_do_suck_it_up_or_fatfire/
[95/100]18 comments from /r/fatFIRE/comments/9v86ku/a_tale_of_two_annuities/
[96/100]39 comments from /r/fatFIRE/comments/9v8kjl/hit_a_gold_mine_and_am_now_making_15kday_where/
[97/100]23 comments from /r/fatFIRE/comments/9v6aew/did_your_wardrobe_change_after_reaching_fatfire/
[98/100]37 comments from /r/fatFIRE/comments/9uphuv/fatfire_missing_in_mainstream_media_articles/
[99/100]13 comments from /r/fatFIRE/comments/9uhdp5/health_insurance_when_early_retired_option/
[100/100]20 comments from /r/fatFIRE/comments/9uiz41/power_and_influence_af

[81/100]14 comments from /r/fatFIRE/comments/9jyjr0/planning_for_bridge_years_between_retirement_and/
[82/100]46 comments from /r/fatFIRE/comments/9jqkke/fatfire_in_a_high_tax_country/
[83/100]58 comments from /r/fatFIRE/comments/9jozzz/high_yield_savings_accounts_a_few_ive_found/
[84/100]60 comments from /r/fatFIRE/comments/9jkbhg/us_citizen_fatfire_abroad_how_to_maintain/
[85/100]26 comments from /r/fatFIRE/comments/9j9sc6/reasonable_yield_in_retirement/
[86/100]20 comments from /r/fatFIRE/comments/9j9hg4/looking_for_coopresident_for_my_fast_growing/
[87/100]12 comments from /r/fatFIRE/comments/9j8zfx/thoughts_on_alternative_investment_platforms_like/
[88/100]33 comments from /r/fatFIRE/comments/9j3cbv/house_prices_in_hcol_cities/
[89/100]34 comments from /r/fatFIRE/comments/9iu702/529_vs_taxable_accounts/
[90/100]71 comments from /r/fatFIRE/comments/9illoc/for_the_fatfire_families_at_what_point_if_ever_do/
[91/100]11 comments from /r/fatFIRE/comments/9iib04/excess_cash_in_business_a

[71/100]40 comments from /r/fatFIRE/comments/99nofc/13m_from_acquisition_of_company_looking_for_a_gut/
[72/100]24 comments from /r/fatFIRE/comments/99e9ej/debating_moving_investments_from_managed_account/
[73/100]3 comments from /r/fatFIRE/comments/996j1g/an_analysis_of_charitable_giving_using_donor/
[74/100]33 comments from /r/fatFIRE/comments/98zeom/perhaps_a_different_kind_of_questionhow_would_you/
[75/100]29 comments from /r/fatFIRE/comments/98mvqe/personal_accounting_software_recommendations/
[76/100]27 comments from /r/fatFIRE/comments/98ku93/is_fatfire_your_goal_or_is_it_just_a_backup_plan/
[77/100]38 comments from /r/fatFIRE/comments/98gglp/what_less_wellknown_tax_moves_are_worth_the_effort/
[78/100]27 comments from /r/fatFIRE/comments/98brwl/meeting_with_life_insurance_salesman_what/
[79/100]39 comments from /r/fatFIRE/comments/982svq/401k_provider_wont_allow_mega_backdoor_roth_what/
[80/100]79 comments from /r/fatFIRE/comments/97yeg9/can_adding_rental_properties_to_your_portf

[60/100]24 comments from /r/fatFIRE/comments/8xtg6q/considering_moving_to_the_bay_area_but_quality_of/
[61/100]82 comments from /r/fatFIRE/comments/8xpqwi/protecting_assets_during_a_divorce/
[62/100]49 comments from /r/fatFIRE/comments/8xqm4j/what_expenses_make_fatfire_better_than_leanfire/
[63/100]6 comments from /r/fatFIRE/comments/8xre58/folks_who_purchased_large_properties_how_did_you/
[64/100]3 comments from /r/fatFIRE/comments/8xq7d6/balance_of_tax_advantaged_assets/
[65/100]10 comments from /r/fatFIRE/comments/8xopj6/healthcare_for_fatfire/
[66/100]8 comments from /r/fatFIRE/comments/8xodr8/any_tips_on_plan_for_new_grad/
[67/100]25 comments from /r/fatFIRE/comments/8xhfqw/19_years_old_starting_out_looking_for_some_ideas/
[68/100]25 comments from /r/fatFIRE/comments/8xldoh/market_timing/
[69/100]10 comments from /r/fatFIRE/comments/8xfx71/bay_area_re_meetup/
[70/100]22 comments from /r/fatFIRE/comments/8x2mzm/tips_for_poor_20_yo/
[71/100]50 comments from /r/fatFIRE/comments/8wllx

[51/100]12 comments from /r/fatFIRE/comments/8jh6ng/fatfire_dads_life_lessons_for_his_daughters/
[52/100]10 comments from /r/fatFIRE/comments/8jgxtx/keeping_track_of_it_all/
[53/100]72 comments from /r/fatFIRE/comments/8ik9y5/asking_the_made_it_and_aspiring_crowd_lets_talk/
[54/100]26 comments from /r/fatFIRE/comments/8i7o5g/a_fatfire_investment_that_can_be_cashed_out/
[55/100]9 comments from /r/fatFIRE/comments/8i1cx0/meta_sidebar_description/
[56/100]20 comments from /r/fatFIRE/comments/8i08ki/meta_anyone_want_to_be_added_as_a_mod/
[57/100]18 comments from /r/fatFIRE/comments/8hyfg1/28mnw675kcan/
[58/100]53 comments from /r/fatFIRE/comments/8hs1u2/how_fat_for_fatfire/
[59/100]14 comments from /r/fatFIRE/comments/8hrrbl/recent_graduate_weighing_options/
[60/100]21 comments from /r/fatFIRE/comments/8hixwu/whats_your_asset_protection_strategy/
[61/100]24 comments from /r/fatFIRE/comments/8hgdwf/fulltime_travel_postfatfire_friendships/
[62/100]19 comments from /r/fatFIRE/comments/8h6syj/

[42/100]15 comments from /r/fatFIRE/comments/83i60c/question_about_affiliate_marketing/
[43/100]15 comments from /r/fatFIRE/comments/83axg7/retiree_w_4_million_in_commercial_real_estate/
[44/100]16 comments from /r/fatFIRE/comments/831lxb/a_quick_demonstration_on_the_power_of_compound/
[45/100]42 comments from /r/fatFIRE/comments/82qo6l/seeking_advice_on_how_to_meet_other_fat_folks/
[46/100]17 comments from /r/fatFIRE/comments/82gkty/what_are_your_thoughts_on_life_insurance_and_how/
[47/100]90 comments from /r/fatFIRE/comments/82055e/on_owning_expensive_things/
[48/100]29 comments from /r/fatFIRE/comments/81t0xn/so_22_and_i_25_are_starting_out_our_lives/
[49/100]19 comments from /r/fatFIRE/comments/81hy1q/lowest_cost_way_to_borrow/
[50/100]33 comments from /r/fatFIRE/comments/80kvii/roth_question/
[51/100]36 comments from /r/fatFIRE/comments/80a4n3/anyone_with_kids_currently_in_private_school/
[52/100]30 comments from /r/fatFIRE/comments/8003g8/the_fatfire_outlook_on_lifestyle_inflatio

[33/100]13 comments from /r/fatFIRE/comments/78345j/achieving_the_initial_capital_to_start_a_business/
[34/100]21 comments from /r/fatFIRE/comments/77wcm2/valuing_smaller_amounts_of_money_again/
[35/100]20 comments from /r/fatFIRE/comments/77u4mo/24m_starting_my_fatfire_journey_appreciate_all/
[36/100]44 comments from /r/fatFIRE/comments/77f0ub/do_you_feel_guilty_about_being_wealthy_do_you/
[37/100]4 comments from /r/fatFIRE/comments/77hi6a/defined_benefit_plans_profit_sharing/
[38/100]8 comments from /r/fatFIRE/comments/773gji/what_it_costs_to_live_well_in_the_us/
[39/100]13 comments from /r/fatFIRE/comments/773zvg/other_peoples_money/
[40/100]7 comments from /r/fatFIRE/comments/76v5b7/new_comer_to_fatfire/
[41/100]51 comments from /r/fatFIRE/comments/76ne8m/what_are_the_best_spend_money_to_make_money/
[42/100]10 comments from /r/fatFIRE/comments/75pa3b/retiring_young_need_financial_advice/
[43/100]17 comments from /r/fatFIRE/comments/74sriv/major_milestone_today/
[44/100]21 comments 

[25/64]22 comments from /r/fatFIRE/comments/5vo8pi/what_would_you_change_between_fire_and_fatfire/
[26/64]27 comments from /r/fatFIRE/comments/5vlb9e/how_much_are_you_spending_on_your_kids/
[27/64]7 comments from /r/fatFIRE/comments/5vdl1w/folks_currently_enjoying_fatfire_what_are_you_up/
[28/64]13 comments from /r/fatFIRE/comments/5u2npk/unmarried_people_how_do_you_balance_your_spending/
[29/64]17 comments from /r/fatFIRE/comments/5tv3vc/did_you_become_fi_then_re_then_grew_wealth_so_you/
[30/64]40 comments from /r/fatFIRE/comments/5tq6vp/do_you_fly_business_class/
[31/64]12 comments from /r/fatFIRE/comments/5sm5vk/best_caribbean_island_to_retire_to/
[32/64]91 comments from /r/fatFIRE/comments/5rzkgm/evaluate_my_plan_26m_us_13mm_net/
[33/64]9 comments from /r/fatFIRE/comments/5rywkz/summer_vacation_plans/
[34/64]36 comments from /r/fatFIRE/comments/5rtsl3/at_what_point_of_wealth_development_does_the_need/
[35/64]8 comments from /r/fatFIRE/comments/5pzkk0/fatfire_in_canada_ontario_not_t

In [23]:
df1 = scrape(subs[1])

[1/101]18 comments from /r/leanfire/comments/a6zb6b/weekly_leanfire_discussion_december_17_2018/
[2/101]4 comments from /r/leanfire/comments/a8ccll/sureproof_early_access_to_401k/
[3/101]17 comments from /r/leanfire/comments/a87zuf/ttfi_bill_introduced_today_great_news_for_us/
[4/101]22 comments from /r/leanfire/comments/a7c1nk/update_easing_my_foot_off_the_gas/
[5/101]15 comments from /r/leanfire/comments/a7fkaa/absolute_beginner_questions/
[6/101]11 comments from /r/leanfire/comments/a7hxah/anyone_consider_a_heloc_to_pull_out_cash_to/
[7/101]17 comments from /r/leanfire/comments/a76id8/hey_everybody_absolute_beginner_to_this_concept/
[8/101]37 comments from /r/leanfire/comments/a6y88g/staying_the_course_in_stressful_jobs/
[9/101]27 comments from /r/leanfire/comments/a70e03/is_it_worth_doing_a_traditional_ira_vs_a_roth_ira/
[10/101]16 comments from /r/leanfire/comments/a7bkm8/what_is_your_suggestion_for_a_good_portfolio/
[11/101]163 comments from /r/leanfire/comments/a6m4b2/leanfired_

[88/101]52 comments from /r/leanfire/comments/9vln1s/hypothetical_tax_questions/
[89/101]36 comments from /r/leanfire/comments/9vdlo5/bipolar_disability_fi/
[90/101]30 comments from /r/leanfire/comments/9uh6vr/has_anyone_been_dead_set_on_quitting_and_then/
[91/101]28 comments from /r/leanfire/comments/9uijaz/timing_the_market_went_to_an_online_trading/
[92/101]17 comments from /r/leanfire/comments/9ud0de/weekly_leanfire_discussion_november_05_2018/
[93/101]23 comments from /r/leanfire/comments/9u930j/how_do_you_plan_to_receive_income_before_the/
[94/101]8 comments from /r/leanfire/comments/9ubyd6/planning_to_fire/
[95/101]135 comments from /r/leanfire/comments/9ttijx/110k_cash_should_i_invest_it_all_in_vtsax_or_wait/
[96/101]38 comments from /r/leanfire/comments/9tabu6/part_time_abroad/
[97/101]150 comments from /r/leanfire/comments/9t6a5k/was_previously_broke_now_making_105kyr_living_in/
[98/101]7 comments from /r/leanfire/comments/9t82wk/58yr_old_planning_to_retire_in_2yrs_money_in_i

[76/100]54 comments from /r/leanfire/comments/9i21a0/married_conflicting_leanfire_goals/
[77/100]39 comments from /r/leanfire/comments/9htygb/is_dipping_into_and_out_of_the_labor_force/
[78/100]25 comments from /r/leanfire/comments/9hvgv9/where_do_i_stand_financially_and_looking_for_some/
[79/100]35 comments from /r/leanfire/comments/9hsopv/confused_about_iras/
[80/100]99 comments from /r/leanfire/comments/9gyr06/issues_sipping_your_employers_koolaid/
[81/100]59 comments from /r/leanfire/comments/9gwt5w/coast_fire_5_month_update/
[82/100]11 comments from /r/leanfire/comments/9gztd2/rental_properties_and_taxes/
[83/100]10 comments from /r/leanfire/comments/9gy0qc/drop_vs_retirement/
[84/100]4 comments from /r/leanfire/comments/9guo9m/somebody_in_pf_needs_your_help/
[85/100]195 comments from /r/leanfire/comments/9gme0g/what_advice_would_you_have_given_your_19_year_old/
[86/100]22 comments from /r/leanfire/comments/9gjstc/weekly_leanfire_discussion_september_17_2018/
[87/100]22 comments f

[66/100]36 comments from /r/leanfire/comments/944nf5/fidelity_launches_two_zero_expense_index_funds/
[67/100]13 comments from /r/leanfire/comments/946jek/nearing_the_finish_line_i_think_and_i_would/
[68/100]3 comments from /r/leanfire/comments/947ywr/heres_my_income_report_for_july_2018_fire/
[69/100]15 comments from /r/leanfire/comments/944xmr/seasonal_work_or_offon_work_as_a_bridge_to_fire/
[70/100]23 comments from /r/leanfire/comments/942bj3/has_anyone_dealt_with_this_aca_vs_tax/
[71/100]76 comments from /r/leanfire/comments/93tp24/whats_with_dave_ramsey_and_the_12_return_every/
[72/100]81 comments from /r/leanfire/comments/93talf/id_love_to_quit_and_pursue_a_dreamjob_as_an/
[73/100]32 comments from /r/leanfire/comments/93ua7n/energy_drinks_and_groceries644_monthly/
[74/100]38 comments from /r/leanfire/comments/93isvd/leanfire_away/
[75/100]29 comments from /r/leanfire/comments/9394qb/leanfire_abroadbut_taxes/
[76/100]48 comments from /r/leanfire/comments/933k4n/weekly_leanfire_disc

[55/100]26 comments from /r/leanfire/comments/8s9boj/understanding_leanfire_more/
[56/100]15 comments from /r/leanfire/comments/8schpc/what_one_year_educational_program_can_give_you_an/
[57/100]11 comments from /r/leanfire/comments/8s1doh/trying_to_figure_out_my_fire_numbers/
[58/100]45 comments from /r/leanfire/comments/8rz1dj/weekly_leanfire_discussion_june_18_2018/
[59/100]54 comments from /r/leanfire/comments/8rtwff/leanfireing_with_201334_any_advice/
[60/100]33 comments from /r/leanfire/comments/8rniue/financial_flowchart_for_fi/
[61/100]30 comments from /r/leanfire/comments/8rkg13/dying_at_your_desk_is_not_a_retirement_plan/
[62/100]7 comments from /r/leanfire/comments/8rkl5d/retiring_in_greece/
[63/100]29 comments from /r/leanfire/comments/8rifpz/leanfire_and_inflation/
[64/100]78 comments from /r/leanfire/comments/8rcdv5/q_whats_a_cheap_fun_vacation_staycation_idea/
[65/100]26 comments from /r/leanfire/comments/8rbqvv/cashand_what_to_do_with_it/
[66/100]13 comments from /r/lean

...

[24/100]80 comments from /r/leanfire/comments/7mu7ai/hypothetical_situation_lets_say_you_came_into_a/
[25/100]13 comments from /r/leanfire/comments/7mnf02/harder_to_work_the_closer_you_get/
[26/100]17 comments from /r/leanfire/comments/7mrass/budget_tool_idea/
[27/100]21 comments from /r/leanfire/comments/7mnr5t/what_about_healthcare_in_early_retirement/
[28/100]7 comments from /r/leanfire/comments/7mpm2q/looking_for_ira_advice/
[29/100]46 comments from /r/leanfire/comments/7me6o9/married_and_i_have_no_friends_seriously_where_do/
[30/100]32 comments from /r/leanfire/comments/7m6jf3/am_i_understanding_the_trinity_study_correctly/
[31/100]21 comments from /r/leanfire/comments/7m59pk/fastest_way_to_bare_minimum_self_subsistence/
[32/100]55 comments from /r/leanfire/comments/7m140g/weekly_leanfire_discussion_december_25_2017/
[33/100]52 comments from /r/leanfire/comments/7lv1f9/am_i_actually_fi_or_am_i_missing_something/
[34/100]18 comments from /r/leanfire/comments/7lwg2i/examining_the_co

[12/100]31 comments from /r/leanfire/comments/74s42w/inheriting_a_house_next_year_what_should_i_do/
[13/100]35 comments from /r/leanfire/comments/74jb87/splurging_when_traveling/
[14/100]49 comments from /r/leanfire/comments/74ep1v/how_long_can_you_live_with_roommates/
[15/100]90 comments from /r/leanfire/comments/745ke9/how_much_money_do_you_consider_to_be_fu_money/
[16/100]33 comments from /r/leanfire/comments/73zya6/weekly_leanfire_discussion_october_03_2017/
[17/100]33 comments from /r/leanfire/comments/73s0gi/getting_paid_to_cycle_with_ubereats/
[18/100]9 comments from /r/leanfire/comments/73s8p5/focusing_on_gross_expenses_vs_relative_expenses/
[19/100]36 comments from /r/leanfire/comments/73lrf4/how_to_prioritize_investing_as_a_very_low_income/
[20/100]6 comments from /r/leanfire/comments/73kwe7/1st_rleanfire_progress_report_post/
[21/100]38 comments from /r/leanfire/comments/73fnug/am_i_on_track_for_leanfire/
[22/100]49 comments from /r/leanfire/comments/7388je/just_a_question_w

[1/100]39 comments from /r/leanfire/comments/6fjyd6/fire_and_housing/
[2/100]9 comments from /r/leanfire/comments/6f6ztr/we_are_starting_reuropefire_a_subreddit_for/
[3/100]3 comments from /r/leanfire/comments/6exdni/continue_funding_401k_or_fund_runway_in_retail/
[4/100]28 comments from /r/leanfire/comments/6e6xky/weekly_leanfire_discussion_may_30_2017/
[5/100]26 comments from /r/leanfire/comments/6e4slr/early_days_leanfire_log_another_500us_month/
[6/100]17 comments from /r/leanfire/comments/6e0vq6/disadvantages_of_putting_money_in_401kira_if/
[7/100]28 comments from /r/leanfire/comments/6dtamh/should_i_quit_my_job_well_before_fire/
[8/100]19 comments from /r/leanfire/comments/6dokta/simple_life_or_keep_pushing/
[9/100]24 comments from /r/leanfire/comments/6dhg8z/do_you_need_emergency_fund_saved_up_during_the/
[10/100]18 comments from /r/leanfire/comments/6d3yhb/a_canadian_leanfire_article/
[11/100]26 comments from /r/leanfire/comments/6cw6ir/wanted_to_share_my_story_just_crossed_100

[92/100]19 comments from /r/leanfire/comments/5u1i4i/enjoying_a_postfire_fire/
[93/100]8 comments from /r/leanfire/comments/5tztk9/weekly_leanfire_discussion_february_14_2017/
[94/100]55 comments from /r/leanfire/comments/5tw7fq/after_7_long_years_of_paying_pmi_on_my_mortgage/
[95/100]5 comments from /r/leanfire/comments/5tqx68/currently_free_book_of_possible_interest_expat/
[96/100]36 comments from /r/leanfire/comments/5t8a3u/a_thought_experiment_a_fire_suburban_minicommune/
[97/100]109 comments from /r/leanfire/comments/5sqzgs/is_anyone_else_here_starting_from_the_bottom/
[98/100]21 comments from /r/leanfire/comments/5suawu/inflation_and_leanfire/
[99/100]20 comments from /r/leanfire/comments/5smvk3/plan_realistic_canadian_couple/
[100/100]146 comments from /r/leanfire/comments/5sho3a/i_find_a_lot_of_the_popular_fi_people_to_be/
Fetching next 100 posts with after key: t3_5sho3a
[1/98]31 comments from /r/leanfire/comments/5sl0jd/weekly_leanfire_discussion_february_07_2017/
[2/98]11 co

[80/98]41 comments from /r/leanfire/comments/57r0y1/if_you_had_the_option_to_bank_your_overtime_for/
[81/98]51 comments from /r/leanfire/comments/56xn9v/weekly_leanfire_discussion_october_11_2016/
[82/98]11 comments from /r/leanfire/comments/55xrz0/457b_vs_403b/
[83/98]43 comments from /r/leanfire/comments/55t5da/weekly_leanfire_discussion_october_04_2016/
[84/98]53 comments from /r/leanfire/comments/55n1tx/retire_for_500month_in_thailand/
[85/98]5 comments from /r/leanfire/comments/55iniw/advice_or_resources_for_investing_and_withdrawing/
[86/98]38 comments from /r/leanfire/comments/55bfeb/this_months_expenses_552us/
[87/98]19 comments from /r/leanfire/comments/559ioi/xpost_from_rfinancialindependence_nearly_60_of/
[88/98]41 comments from /r/leanfire/comments/550nc8/leanfire_and_a_social_life/
[89/98]38 comments from /r/leanfire/comments/54q89x/weekly_leanfire_discussion_september_27_2016/
[90/98]14 comments from /r/leanfire/comments/54ppmi/what_are_your_favourite_fire_blogs/
[91/98]2

In [24]:
df0.columns

Index(['approved_at_utc', 'approved_by', 'archived', 'author',
       'author_cakeday', 'author_flair_background_color',
       'author_flair_css_class', 'author_flair_richtext',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_patreon_flair', 'banned_at_utc', 'banned_by', 'can_gild',
       'can_mod_post', 'category', 'clicked', 'comments', 'content_categories',
       'contest_mode', 'created', 'created_utc', 'crosspost_parent',
       'crosspost_parent_list', 'distinguished', 'domain', 'downs', 'edited',
       'gilded', 'gildings', 'hidden', 'hide_score', 'id', 'is_crosspostable',
       'is_meta', 'is_original_content', 'is_reddit_media_domain',
       'is_robot_indexable', 'is_self', 'is_video', 'likes',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
       'link_flair_text_color', 'link_flair_typ

In [9]:
df0.head()

,approved_at_utc,approved_by,archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,...,thumbnail_height,thumbnail_width,title,ups,url,user_reports,view_count,visited,whitelist_status,wls
0,None,None,False,leedar1376,NaN,None,None,[],None,None,...,NaN,NaN,Advisor fees at various asset levels?,20,https://www.reddit.com/r/fatFIRE/comments/a815...,[],None,False,None,None
1,None,None,False,a_random_tomato,NaN,None,None,[],None,None,...,NaN,NaN,HSA record keeping,39,https://www.reddit.com/r/fatFIRE/comments/a7s7...,[],None,False,None,None
2,None,None,False,retiringearly,NaN,None,None,[],None,None,...,NaN,NaN,2019 Money Goals,79,https://www.reddit.com/r/fatFIRE/comments/a7f3...,[],None,False,None,None
3,None,None,False,kernelcrop,NaN,None,None,[],None,None,...,NaN,NaN,What’s is your Tax Loss Harvesting Threshold?,15,https://www.reddit.com/r/fatFIRE/comments/a7fu...,[],None,False,None,None
4,None,None,False,TreesButterPanny,NaN,None,None,[],None,None,...,NaN,NaN,"Might be retiring earlier than planned, hoping...",40,https://www.reddit.com/r/fatFIRE/comments/a76b...,[],None,False,None,None


### Save subreddit posts separately

In [10]:
df0.to_csv(f'../data/{subs[0]}.csv', index = False)
df1.to_csv(f'../data/{subs[1]}.csv', index = False)

### Combine the data, shuffle and save it

In [11]:
df = pd.concat([df0, df1], axis = 0)
df = df.sample(frac = 1, random_state = 42).reset_index(drop = True)

/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [12]:
df['subreddit'].value_counts()

leanfire    999
fatFIRE     764
Name: subreddit, dtype: int64

The classes are sufficiently balanced.

In [26]:
df['subreddit'].value_counts(normalize = True)

leanfire    0.566648
fatFIRE     0.433352
Name: subreddit, dtype: float64

In [27]:
df.to_csv(f'../data/{subs[1]}-{subs[0]}.csv', index = False)